In [18]:
import pyswarms as ps
import random
from read_data import read_dataset_1, translate_1_testing
from pyswarms.utils.functions import single_obj as fx
from models import SimulationEnvironment

In [19]:
# Set-up hyperparameters
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
# Call instance of PSO
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=2, options=options)
# Perform optimization
best_cost, best_pos = optimizer.optimize(fx.rastrigin, iters=100)

2022-09-26 15:26:01,208 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=0.995
2022-09-26 15:26:01,309 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.9949919903178142, best pos: [ 9.95334646e-01 -1.57116372e-04]


In [20]:
order_amount = 5
earliest_time_slot = 0
last_time_slot = 5000 # shouldn't actually be necessary for this

In [21]:
input, orders, instance = read_dataset_1(use_instance=6, order_amount=order_amount, earliest_time=earliest_time_slot, last_time=last_time_slot)
#recipes, workstations, resources, tasks, orders_model = translate_1(instance, orders, earliest_time_slot, last_time_slot)
recipes, workstations, resources, tasks, orders_model = translate_1_testing(instance, orders, earliest_time_slot, last_time_slot)
env = SimulationEnvironment(workstations, tasks, resources, recipes)

In [ ]:
bounds = (earliest_time_slot, last_time_slot)
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
# build input array
assignments = []
start_times = []
associated_orders = []
duration_lookup_table = dict()
for task in tasks:
    if not task.external_id in duration_lookup_table:
            duration_lookup_table[task.external_id] = dict()
    for workstation in env.get_valid_workstations(task.external_id):
        duration_lookup_table[task.external_id][workstation.id] = env.get_duration(task.external_id, workstation.id)

operations = []
order_for_index = []
for order in orders_model:
    for resource in order.resources:
        recipe = resource.recipes[0] # just use recipe 0 for now
        recipe_tasks = env.get_all_tasks_for_recipe(recipe.id)
        results = dict()
        for task in recipe_tasks:
            if task.result_resources[0][0] not in results:
                results[task.result_resources[0][0]] = []
            results[task.result_resources[0][0]].append(task)
        for key in results:
            #operations.append(random.choice(results[key]).id)
            operations.append(random.choice(results[key]).external_id)
            order_for_index.append(order.id)

for operation in operations:
    workstation = env.get_valid_workstations(operation)
    # random init
    assignments.append(random.choice(workstation).id)
    assignments.append(0) # start time slot

# TODO: remove start times from encoding
def calculate_start_time():
    pass

# feasibility function
def feasible(x):
    for i in range(0, len(x), 2):
        # check if workstation is valid for task
        if x[i] not in duration_lookup_table[operations[int(i / 2)]]: 
            return False
        # check if time is within bounds
        if x[i+1] + duration_lookup_table[operations[int(i / 2)]][x[i]] > last_time_slot:
            return False
        if x[i+1] < 0:
            return False
        # TODO: check for overlaps

# objective function - using makespan for now
def objective_function(x):
    if feasible(x):
        min = float('inf')
        max = 0
        for i in range(0, x, 2):
            if x[i+1] < min:
                min = x[i+1]
            if x[i+1] + duration_lookup_table[operations[int(i / 2)]][x[i]] > max:
                x[i+1] + duration_lookup_table[operations[int(i / 2)]][x[i]]
        return max - min
    else:
        return float('inf')
        
# setup optimizer
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=len(assignments), options=options, bounds=bounds)
# Perform optimization
cost, pos = optimizer.optimize(objective_function, iters=1000)